In [ ]:
import pandas as pd
import datetime
import os
import pytz
import json
print(pd.__version__)
utc = pytz.utc

dataFileColumns = ["startDate", "endDate", "value"]

path = "/home/chowder/Documents/personalData/ouraRing/"
jsonFileName = "oura_2021-08-10T02-14-44.json"

outputFileName = "ouraRingHypnoDf.parquet.gzip"

with open(path + jsonFileName) as f:
  jsonData = json.load(f)

In [ ]:
def ouraSleepStageToStandard(ouraNum):
    if ouraNum == 4: #awake
        return 0
    if ouraNum == 2: #light sleep
        return 1
    if ouraNum == 1: #deep sleep
        return 2
    if ouraNum == 3: #REM sleep
        return 3
    

In [ ]:
for SleepJSON in jsonData['sleep']:
    startTime = datetime.datetime.fromisoformat(SleepJSON['bedtime_start']).astimezone(utc)
    stageStartTime = startTime

    hypnogram_5min = SleepJSON['hypnogram_5min']